In [ ]:
!pip install cloudscraper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 2.0 MB/s eta 0:00:00


In [ ]:
import pyspark
from pyspark.sql import SparkSession


spark = SparkSession.builder.appName("HelloWorld").getOrCreate()
sc = spark.sparkContext

nums = sc.parallelize([1,2,3,4])
result=nums.map(lambda x: x*x).collect()
print(result)

[1, 4, 9, 16]


In [ ]:
import pandas as pd
import cloudscraper
from bs4 import BeautifulSoup

def scrape_reviews_to_df(perfume_urls):
    scraper = cloudscraper.create_scraper(
        browser={
            'browser': 'chrome',
            'platform': 'windows',
            'mobile': False
        }
    )
    all_reviews = []

    for perfume_url in perfume_urls:
        reviews = []
        page_url = perfume_url

        while page_url:
            resp = scraper.get(page_url)
            resp.raise_for_status()
            soup = BeautifulSoup(resp.text, 'html.parser')

            for box in soup.select('div.fragrance-review-box'):
                author_el = box.select_one('[itemprop="author"] [itemprop="name"]')
                date_el   = box.select_one('[itemprop="datePublished"]')
                perf_el   = box.select_one('.perfume-wearing-icon')
                rate_el   = box.select_one('[rating-vote]')
                text_el   = box.select_one('[itemprop="reviewBody"]')

                reviews.append({
                    'url': perfume_url,
                    'author':      author_el  ['content'] if author_el else None,
                    'date':        date_el    ['content'] if date_el   else None,
                    'perfume':     perf_el    ['title']   if perf_el   else None,
                    'rating':      int(rate_el['rating-vote']) if rate_el else None,
                    'review_text': text_el.get_text(strip=True)   if text_el  else None
                })

            next_link = soup.select_one('ul.pagination li.next a')
            page_url = ('https://www.fragrantica.com.br' + next_link['href']) if next_link else None

        all_reviews.extend(reviews)

    return pd.DataFrame(all_reviews)

if __name__ == '__main__':
    urls = [
        'https://www.fragrantica.com.br/perfume/Carolina-Herrera/Good-Girl-39681.html',
        'https://www.fragrantica.com.br/perfume/Lancome/La-Vie-Est-Belle-14982.html',
        'https://www.fragrantica.com.br/perfume/Yves-Saint-Laurent/Libre-56077.html',
        'https://www.fragrantica.com.br/perfume/Prada/Prada-Paradoxe-75668.html',
        'https://www.fragrantica.com.br/perfume/Jean-Paul-Gaultier/Scandal-45651.html',
        'https://www.fragrantica.com.br/perfume/Giorgio-Armani/My-Way-62036.html',
        'https://www.fragrantica.com.br/perfume/Dior/J-Adore-L-Or-10371.html',
        'https://www.fragrantica.com.br/perfume/Giorgio-Armani/Si-18453.html',
        'https://www.fragrantica.com.br/perfume/Yves-Saint-Laurent/Black-Opium-Eau-de-Toilette-31440.html',
        'https://www.fragrantica.com.br/perfume/Chanel/Coco-Mademoiselle-611.html',
        'https://www.fragrantica.com.br/perfume/Lancome/Idole-55795.html',
        'https://www.fragrantica.com.br/perfume/Dior/Miss-Dior-Eau-de-Parfum-2021-68905.html',
        'https://www.fragrantica.com.br/perfume/Calvin-Klein/Euphoria-253.html',
        'https://www.fragrantica.com.br/perfume/Givenchy/L-Interdit-Eau-de-Parfum-51488.html'

        # Add more URLs as needed
    ]
    df = scrape_reviews_to_df(urls)
    print(df)



                                                    url  \
0     https://www.fragrantica.com.br/perfume/Carolin...   
1     https://www.fragrantica.com.br/perfume/Carolin...   
2     https://www.fragrantica.com.br/perfume/Carolin...   
3     https://www.fragrantica.com.br/perfume/Carolin...   
4     https://www.fragrantica.com.br/perfume/Carolin...   
...                                                 ...   
2456  https://www.fragrantica.com.br/perfume/Givench...   
2457  https://www.fragrantica.com.br/perfume/Givench...   
2458  https://www.fragrantica.com.br/perfume/Givench...   
2459  https://www.fragrantica.com.br/perfume/Givench...   
2460  https://www.fragrantica.com.br/perfume/Givench...   

                     author        date                               perfume  \
0                  Sylvinha  2025-06-06  Valentino Donna Born In Roma Intense   
1                    Mah C.  2025-06-01                                  None   
2                bbrendinha  2025-05-30         

In [ ]:
import cloudscraper
from bs4 import BeautifulSoup

# 1) Fetch the “female 2023” search page
search_url = "https://www.fragrantica.com.br/busca/?godina=2023%3A&spol=female"
scraper = cloudscraper.create_scraper(
    browser={"browser": "chrome", "platform": "windows", "mobile": False}
)
resp = scraper.get(search_url)
resp.raise_for_status()
soup = BeautifulSoup(resp.text, "html.parser")

In [ ]:
perfumes = []
for card in soup.select("div.fr-news-box"):
    # Only select <a> tags whose href begins with "/perfume/"
    a_tag = card.select_one('a[href^="/perfume/"]')
    if not a_tag:
        continue

    href = a_tag["href"]  # e.g. "/perfume/Burberry/Goddess-83483.html"
    if not href.startswith("/perfume/"):
        continue  # skip anything that isn’t a proper perfume link

    full_url = "https://www.fragrantica.com.br" + href
    name = a_tag.get_text(strip=True)

    perfumes.append((name, full_url))
perfumes

[('Azzaro Orange Tonic',
  'https://www.fragrantica.com.br/perfume/Azzaro/Orange-Tonic-1.html'),
 ('Givenchy Amarige',
  'https://www.fragrantica.com.br/perfume/Givenchy/Amarige-3.html'),
 ('Givenchy Organza',
  'https://www.fragrantica.com.br/perfume/Givenchy/Organza-4.html'),
 ("Dior J'adore Eau de Toilette 2002",
  'https://www.fragrantica.com.br/perfume/Dior/J-adore-Eau-de-Toilette-2002-5.html'),
 ('Lanvin Arpège',
  'https://www.fragrantica.com.br/perfume/Lanvin/Arpege-6.html'),
 ('Hermès Equipage',
  'https://www.fragrantica.com.br/perfume/Hermes/Equipage-7.html'),
 ('Hermès Rouge Hermes Eau Delicate',
  'https://www.fragrantica.com.br/perfume/Hermes/Rouge-Hermes-Eau-Delicate-8.html'),
 ('Hermès Eau des Merveilles',
  'https://www.fragrantica.com.br/perfume/Hermes/Eau-des-Merveilles-9.html'),
 ('Hermès Parfum des Merveilles',
  'https://www.fragrantica.com.br/perfume/Hermes/Parfum-des-Merveilles-10.html'),
 ('Chanel Chanel N°19',
  'https://www.fragrantica.com.br/perfume/Chanel/C

In [ ]:
def scrape_reviews_to_df(perfume_urls):
    scraper = cloudscraper.create_scraper(
        browser={
            'browser': 'chrome',
            'platform': 'windows',
            'mobile': False
        }
    )
    all_reviews = []

    for perfume_url in perfume_urls:
        reviews = []
        page_url = perfume_url

        while page_url:
            resp = scraper.get(page_url)
            resp.raise_for_status()
            soup = BeautifulSoup(resp.text, 'html.parser')

            for box in soup.select('div.fragrance-review-box'):
                author_el = box.select_one('[itemprop="author"] [itemprop="name"]')
                date_el   = box.select_one('[itemprop="datePublished"]')
                perf_el   = box.select_one('.perfume-wearing-icon')
                rate_el   = box.select_one('[rating-vote]')
                text_el   = box.select_one('[itemprop="reviewBody"]')

                reviews.append({
                    'url': perfume_url,
                    'author':      author_el  ['content'] if author_el else None,
                    'date':        date_el    ['content'] if date_el   else None,
                    'perfume':     perf_el    ['title']   if perf_el   else None,
                    'rating':      int(rate_el['rating-vote']) if rate_el else None,
                    'review_text': text_el.get_text(strip=True)   if text_el  else None
                })

            next_link = soup.select_one('ul.pagination li.next a')
            page_url = ('https://www.fragrantica.com.br' + next_link['href']) if next_link else None

        all_reviews.extend(reviews)

    return pd.DataFrame(all_reviews)

In [ ]:
all_reviews_df = pd.DataFrame()

for (perfume_name, perfume_url) in perfumes:
    print(f"\n=== Scraping reviews for: {perfume_name} ===")
    df_reviews = scrape_reviews_to_df([perfume_url])
    all_reviews_df = pd.concat([all_reviews_df, df_reviews], ignore_index=True)

print(all_reviews_df.head())


=== Scraping reviews for: Azzaro Orange Tonic ===

=== Scraping reviews for: Givenchy Amarige ===

=== Scraping reviews for: Givenchy Organza ===

=== Scraping reviews for: Dior J'adore Eau de Toilette 2002 ===

=== Scraping reviews for: Lanvin Arpège ===

=== Scraping reviews for: Hermès Equipage ===

=== Scraping reviews for: Hermès Rouge Hermes Eau Delicate ===

=== Scraping reviews for: Hermès Eau des Merveilles ===

=== Scraping reviews for: Hermès Parfum des Merveilles ===

=== Scraping reviews for: Chanel Chanel N°19 ===

=== Scraping reviews for: Chanel Cristalle Eau de Toilette ===

=== Scraping reviews for: Guerlain Apres l'Ondee ===

=== Scraping reviews for: Lanvin Arpege Pour Homme ===

=== Scraping reviews for: Robert Piguet Bandit ===

=== Scraping reviews for: Guerlain Habit Rouge Eau de Toilette ===

=== Scraping reviews for: Hermès Terre d'Hermès ===

=== Scraping reviews for: Hermès Un Jardin Sur Le Nil ===

=== Scraping reviews for: Hermès Un Jardin en Méditerranée

HTTPError: 429 Client Error: Too Many Requests for url: https://www.fragrantica.com.br/perfume/Givenchy/Givenchy-pour-Homme-Blue-Label-38.html

In [ ]:
dados = " ".join(all_reviews_df["review_text"].astype(str))
dados

In [ ]:
import re

chaves = [ "acessório", "acompanhamento", "alongamento", "amadeirado", "amo", "atitude", "autoamor", "autoestima",
    "autoimagem", "autocuidado", "autoconfiança", "autenticidade", "beleza", "blogueira", "bodysplash",
    "bolsa", "bolsas", "celebridades", "Chanel", "charme", "closet", "comprinha", "confiança", "conforto",
    "cosméticos", "cupom", "date", "dancinha", "descoberta", "destino", "Dior", "doce", "e-commerce",
    "elegância", "elegante", "emoção", "energia", "enjoativo", "estampas", "estilo", "exclusividade",
    "experiência", "expressão", "falsificado", "favorito", "felicidade", "feminilidade", "feminino",
    "Floral", "força", "Fraco", "frescura", "Frutado", "glamour", "Glow", "Gucci", "hidratação",
    "identidade", "impressiona", "importado", "influencer", "influenciadora", "instagram", "instagramável",
    "inspiração", "irresistivel", "irresistiveis", "Lancôme", "lembrança", "lifestyle", "lip oil", "look",
    "looks", "Longa", "lugar", "luxo", "maquiagem", "Marcante", "memória", "minimalismo", "moda",
    "momento", "Mugler", "necessaire", "Original", "oriental", "outfit", "passaporte", "pele",
    "personalidade", "poder", "popular", "praia", "prazer", "presente", "presentinho", "produto",
    "protagonismo", "promoção", "refrescante", "relaxamento", "resenha", "Réplica", "rosto", "rotina",
    "Sedutor", "sensação", "sensual", "sensualidade", "serum", "Sexy", "signos", "similar", "skincare",
    "social", "sofisticado", "stories", "suavidade", "tendência", "testador", "TikTok", "toque",
    "transformação", "vibe", "verão", "vício", "viagem", "vitrine", "wishlist", "YSL"]
chaves_upper = [k.upper() for k in chaves]

words = re.findall(r"\b\w+\b", dados)
words

###CONTAGEM DAS PALAVRAS QUE ESCOLHEMOS

In [ ]:
input_rdd = sc.parallelize(words, 3)

def map_to_keyword_count(word):
    w_up = word.upper()
    if w_up in chaves_upper:
        return (w_up, 1)
    else:
        return None

counts_rdd = (
    input_rdd
    .map(map_to_keyword_count)
    .filter(lambda kv: kv is not None)
    .reduceByKey(lambda a, b: a + b)
)

result = counts_rdd.collect()
print(result)

In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

stopwords_pt = set(stopwords.words('portuguese'))

palavras_filtradas = [p for p in words if p.lower() not in stopwords_pt]

print(palavras_filtradas)

###CONTAGEM DE TODAS AS PALAVRAS

In [ ]:
from pyspark import SparkContext

# Inicializa SparkContext
sc = SparkContext.getOrCreate()

# Paraleliza a lista de palavras em 3 partições
input_rdd = sc.parallelize(palavras_filtradas, 3)

# Função para mapear cada palavra para (palavra maiúscula, 1)
def map_to_count(word):
    return (word.upper(), 1)

# Conta as palavras
counts_rdd = (
    input_rdd
    .map(map_to_count)
    .reduceByKey(lambda a, b: a + b)
)

# Coleta resultado
result = counts_rdd.collect()

# Imprime resultado
print(result)

In [ ]:
ordenado = sorted(result, key=lambda x: x[1], reverse=True)
print(ordenado)

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import random

# Função para gerar tons de rosa (R alto, G e B baixos/moderados)
def rosa_color_func(word, font_size, position, orientation, random_state=None, **kwargs):
    r = random.randint(200, 255)    # Vermelho forte
    g = random.randint(0, 100)      # Verde baixo
    b = random.randint(100, 180)    # Azul médio para lilás/rosa
    return f"rgb({r}, {g}, {b})"

frequencies = dict(ordenado)

wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(frequencies)

# Aplicar a função de cor rosa
plt.figure(figsize=(15, 7))
plt.imshow(wordcloud.recolor(color_func=rosa_color_func), interpolation='bilinear')
plt.axis("off")
plt.title("Nuvem de Todas as Palavras da Fragrantica")
plt.show()


###REMOVENDO AS PALAVRAS INDESEJADAS

In [ ]:
result_filtrado = [t for t in ordenado if t[0].upper() not in ["PQ","ALI","ETC","JA","UN","IR","PRECISO", "CONSEGUE","TB","VÃO","CREIO","MIM", "BEM", "PERFUME", "PERFUMES", "NOTAS", "FRAGRÂNCIA", "SINTO", "DIA", "ACHO", "ANOS", "TEMPO", "AROMA", "AINDA", "POIS", "NOTA", "POUCO", "FICA", "GOSTO", "VAI", "AQUI", "NADA", "PROJEÇÃO", "TUDO", "TÃO", "FRASCO", "FORTE", "TER", "SENTI", "ALGO", "LO", "VETIVER", "HORAS", "SENTIR", "USAR", "SAÍDA", "ASSIM", "VERDE", "TERRE", "LEMBRA", "AMARIGE", "AMO", "ACHEI", "PODE", "SEI", "TODO", "USO", "MENOS", "BASTANTE", "REALMENTE", "SIM", "HOJE", "PRIMEIRA", "TODOS", "PORQUE", "MEIO", "PIMENTA", "PERFUMARIA", "DÁ", "PORÉM", "DIFERENTE", "PARECE", "VIDA", "SEMPRE", "MARAVILHOSO", "SOBRE", "DESSE", "VEZ", "OBRA", "FAZ", "DIAS", "LEVE", "TERRA", "MULHER", "SENSAÇÃO", "QUALIDADE", "PESSOAS", "FORMA", "D", "VINTAGE", "OUTRO", "FRESCO", "NOITE", "FUNDO", "QUASE", "DEMAIS", "CLÁSSICO", "ATALCADO", "QUALQUER", "CASA", "TOQUE", "TANTO", "QUENTE", "CADA", "ALGUNS", "SUPER", "LOGO", "5", "COMPREI", "TALVEZ", "APENAS", "APESAR", "GOSTA", "ORGANZA", "FEMININO", "REMETE", "NUNCA", "MUITA", "TODA", "GENTE", "ONDE", "QUANTO", "COISA", "AGORA", "APÓS", "VERSÃO", "NELE", "FIXA", "PERFEITO", "10", "DIZER", "FIM", "EXTREMAMENTE", "ROSA", "ANTES", "VEZES", "OUTROS", "CONSIGO", "CEDRO", "TIPO", "MASCULINO", "ALGUMA", "BOA", "USADO", "EVOLUÇÃO", "SENDO", "ENTÃO", "MESMA", "BLIND", "AMOR", "VC", "NAO", "TUBEROSA", "JUNTO", "AGRADÁVEL", "GRANDE", "TODAS", "PERSONALIDADE", "USEI", "PRIMA", "JARDIM", "ÉPOCA", "PASSA", "MUNDO", "ALGUM", "VEM", "PRIMEIRO", "FINAL", "PROJETA", "DESSA", "ALGUMAS", "HORA", "VER", "BORRIFADA", "FIQUEI", "COMBINA", "DAR", "ASSINATURA", "FICOU", "HOMEM", "DIFÍCIL", "PENA", "MULHERES", "JASMIM", "LADO", "COURO", "CHEIRINHO", "OLFATO", "TRAZ", "CONFORTÁVEL", "FLOR", "OLFATIVA", "SABE", "VOU", "NARIZ", "QUER", "2", "PASSAR", "SIMPLESMENTE", "PRESENTE", "MUITOS", "MUITAS", "ABERTURA", "OCASIÕES", "GOSTAR", "AÍ", "NESSE", "PESSOA", "ABRE", "DELICIOSO", "ESTILO", "FALAR", "MOMENTO", "BASE", "ALÉM", "DEVE", "ÚNICO", "CONTA", "UNS", "COMPRAR", "CARA", "FICAR", "FRAGRÂNCIAS", "RESENHAS", "DESDE", "ACREDITO", "VIBE", "LEVEMENTE", "CORAÇÃO", "MOLHADA", "DEIXA", "MINUTOS", "VALE", "8", "SECO", "RS", "NENHUM", "EDT", "QUENTES", "TORNA", "NATURAL", "BORRIFADAS", "FIGO", "USANDO", "RECOMENDO", "CERTEZA", "UMAS", "DATADO", "CHEGA", "PRESENÇA", "FAZER", "MÃE", "LÁ", "CERTA", "EMBORA", "INÍCIO", "COLEÇÃO", "ASPECTO", "GUERLAIN", "BOMBA", "CLIMA", "OPINIÃO", "ADORO", "GOSTEI", "VOLTA", "CONSIDERO", "PREÇO", "MARCA", "ARTE", "CONHECER", "DEI", "Q", "PROPOSTA", "PEGADA", "EXCELENTE", "3", "ALGUÉM", "ROCHAS", "CASO", "DECANT", "NOME", "CABEÇA", "ATEMPORAL", "LUGAR", "ANO", "DIRIA", "OUTRAS", "COMBINAÇÃO", "AR", "LIMÃO", "PARECIDO", "CONHECI", "EDP", "MUSGO", "PRO", "INTERESSANTE", "BANHO", "RESENHA", "PODER", "BRASIL", "TOM", "LINHA", "DUAS", "PERFEITAMENTE", "MISTURA", "AGRADAR", "AMARGA", "YLANG", "DISSO", "ÓTIMO", "MADURO", "SENTE", "SENTIDO", "COMUM", "POSSUI", "LEMBRO", "PARTE", "COISAS", "LE", "BRANCAS", "N", "MODERADA", "JARDIN", "MELHORES", "OCASIÃO", "PONTO", "LOJA", "NOVO", "HOT", "SOL", "IDADE", "FRESCOR", "VEJO", "POTENTE", "CONSEGUI", "USA", "CONHEÇO", "GARDÊNIA", "POSSO", "TRABALHO", "VERDADE", "CAUSA", "ENFIM", "FATO", "QUERO", "USÁ", "ENTENDER", "CHUVA", "FÁCIL", "COMIGO", "CLARO", "SÉRIO", "INFELIZMENTE", "AGRADA", "VI", "NESTE", "DELICADO", "ELOGIOS", "OUTRA", "ROUGE", "MIMOSA", "DENTRO", "SOMENTE", "CORPO", "ÓTIMA", "TERROSO", "BRANCO", "FELIZ", "DAQUELES", "DOIS", "USAVA", "RASTRO", "ÁGUA", "FRESCA", "COMEÇA", "SÂNDALO", "NINGUÉM", "ESPECIAL", "CHEIRA", "SIMPLES", "COMPOSIÇÃO", "ELLENA", "DESTE", "FEITO", "NENHUMA", "CENOURA", "AROMAS", "VIOLETA", "FRIO", "AMBIENTE", "FRUTAS", "DEU", "VELHA", "ADULTO", "HERMÉS", "VELHO", "PERTO", "SIDO", "ORIGINAL", "APARECE", "FOLHA", "DEFINITIVAMENTE", "HOMENS", "PERFORMANCE", "DELÍCIA", "PRECISA", "COMPRA", "TOTALMENTE", "ROUPA", "MADEIRAS", "RUIM", "DANDO", "DISSE", "SECAGEM", "DURANTE", "TARDE", "COMPLETAMENTE", "LONGE", "SI", "MEMÓRIA", "CONFESSO", "9", "SAIR", "COR", "PODEROSO", "DIZ", "NESSA", "ATRÁS", "COUTURE", "RESPEITO", "DÚVIDA", "PULSO", "LIMPO", "EXISTE", "MEDIDA", "90", "L", "POUCOS", "RENTE", "4", "EXPERIÊNCIA", "CERTO", "MARAVILHOSA", "1", "ESPECIADO", "CAUSAR", "PASSADO", "ASSABONETADO", "DEIXANDO", "AI", "DURAÇÃO", "JEITO", "MAISON", "TOMATE", "ENQUANTO", "QUESTÃO", "ANTIGO", "LARANJEIRA", "HABIT", "JEAN", "ALTA", "ALEGRE", "FRIOS", "ATENÇÃO", "DESDEQUE", "NAQUELE", "QUAL", "CUIDADO", "CHEGOU", "VOLUME", "PRAZER", "A", "ENCONTRAR", "VIRA", "CHEGANDO", "EXPERIÊNCIAS", "EXAGERADO", "FALANDO", "PROFUNDIDADE", "CONTINUA", "INTENSO", "AIRES", "POEIRA", "TIPOS", "ESTOU", "TEXTO", "JÁ", "QUEBRA", "FALA", "TANTOFA", "MELHOR", "MUITO", "GOSTARIA", "OUI", "TIPOLOGIA", "CÁLIDO", "SEMANA", "COLONIA", "DEVERIA", "VIU", "NÃO", "LONGAS", "MOUSSE", "NATUREZA", "DIVERSAS", "HOUVE", "MATIZ", "VOLTO", "BATOM", "BOM", "DEMANDA", "TEXTO", "AMBIENTAL", "INTENSIDADE", "PERFUMES", "RICO", "FONTE", "PRA", "MODERADO", "CORAÇÕES", "PROVAVELMENTE", "MAIS", "QUERIA", "BALEIA", "EXCELÊNCIA", "RECEBI", "RASCUNHO", "LUXO", "TRADIÇÃO", "TARDE", "PODERIA", "DIZER", "MORRER", "VIVO", "ELEGANTE", "RAIZ", "PRONTO", "ANDEI", "SOLTO", "ROUPAS", "SOLAR", "SUL", "PESADA", "FALTA", "PEDAÇO", "VISUAL", "ESTILO", "TRADIÇÃO", "DEPOIS", "CHAMAR", "O", "PEQUENA", "NOVA", "GOSTO", "ACONTECE", "MAR", "MUITO", "SEMANA", "SIM", "CINZA", "FELICIDADE", "VEM", "LINDO", "TRÁS", "MARCANTE", "INTERESSANTE", "ÚNICA", "INTENSO", "DAS", "PORQUE", "ESCOLA", "MESMA", "DIZER", "VOCÊ", "HÁ", "SOBRE", "VEZ", "AÍ", "OLHA", "DIZER", "HORA", "PODER", "NADA", "DE", "SER", "HÁ", "TER", "ESTÁ", "FUI", "SER", "SEI", "TEM", "FAZ", "ESTÁ", "COM", "SÓ", "MAIS", "SE", "POR", "TUDO", "NÃO", "MINHA", "SUA", "NOSSA", "DELES", "DELAS", "QUEM", "QUANDO", "ONDE", "PORQUE", "COMO", "QUE", "É", "OU", "MAS", "SE", "ENTÃO", "POIS", "OU", "E", "NEM", "OU", "ENTRE", "EM", "DE", "PARA", "COM", "SEM", "SOB", "APÓS", "ANTES", "DURANTE", "ATRAVÉS", "DENTRO", "FORA", "PERTO", "LONGE", "PRÓXIMO", "ALÉM", "EXCETO", "SALVO", "MENOS", "CONFORME", "SEGUNDO", "ACIMA", "ABAIXO", "ATRÁS", "PERANTE", "CONTRA", "ENTRE", "SOBRE", "POR", "DENTRO", "EM", "PARA", "COM", "SEM", "SOB", "APÓS", "ANTES", "DURANTE", "ATRAVÉS", "DENTRO", "FORA", "PERTO", "LONGE", "PRÓXIMO", "ALÉM", "EXCETO", "SALVO", "MENOS", "CONFORME", "SEGUNDO", "ACIMA", "ABAIXO", "ATRÁS", "PERANTE", "CONTRA"]]

print(result_filtrado)


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import random

def marrom_pastel_color_func(word, font_size, position, orientation, random_state=None, **kwargs):
    # Tons marrons pastéis suaves
    pastel_browns = [
        (210, 180, 140),  # Tan (bege amarronzado)
        (222, 184, 135),  # Burlywood (caramelo claro)
        (244, 164, 96),   # Sandy brown
        (205, 133, 63),   # Peru (marrom claro)
        (193, 154, 107),  # bege médio amarronzado
        (222, 210, 170),  # bege claro amarelado
        (230, 216, 181),  # bege pastel
    ]
    r, g, b = random.choice(pastel_browns)
    return f"rgb({r}, {g}, {b})"

frequencies = dict(result_filtrado)

wordcloud = WordCloud(width=800, height=400, background_color='#f5f0e6').generate_from_frequencies(frequencies)  # fundo bege claro

plt.figure(figsize=(15, 7))
plt.imshow(wordcloud.recolor(color_func=marrom_pastel_color_func), interpolation='bilinear')
plt.axis("off")
plt.title("Nuvem de Palavras da Fragrantica", color='#5C3A21', fontsize=15)
plt.show()


In [ ]:
df_result = pd.DataFrame(result_filtrado, columns=["termo", "frequencia"])
print(df_result.sort_values(by="frequencia", ascending=False))

##ANÁLISE DE SENTIMENTO

In [ ]:
!pip install unidecode
from unidecode import unidecode
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import random
from pyspark import SparkContext
import pandas as pd

In [ ]:
# Inicializa o SparkContext
sc = SparkContext.getOrCreate()

palavras_negativas = ["ruim", "fraco", "enjoativo", "sintético", "desagradável", "agressivo", "dolorento",
                      "irritante", "exagerado", "nauseante", "incomodativo", "artificial", "fadigante",
                      "confuso", "infantil", "barato", "genérico", "ácido", "amargo", "ardido", "pesado",
                      "áspero", "velho", "frio", "morno", "rasteiro", "raso", "vazio", "comum", "repetitivo", "cansativo",
                      "estranho", "químico", "brusco", "apagado", "opaco", "irregular", "instável", "incômodo",
                      "desbotado", "desconfortável", "desinteressante", "desafinado", "desarmônico", "desproporcional",
                      "desgastado", "baixa fixação", "pouca projeção", "doce", "doce demais", "muito doce", "um pouco doce", "muito forte", "muito sintético",
                      "cheiro genérico", "cheiro barato", "cheiro enjoativo", "cheiro estranho", "cheiro desagradável", "cheiro químico",
                      "cheiro artificial", "cheiro infantil", "cheiro fraco", "cheiro apagado", "cheiro comum", "cheiro confuso", "cheiro incômodo",
                      "cheiro agressivo", "cheiro pesado", "muito fraco", "projeção baixa", "baixa fixação", "baixa projeção", "fixação baixa"]

# 2. Dados Coletados
# dados = " ".join(all_reviews_df["review_text"].astype(str))

# 3. Pré-processamento do texto
dados = unidecode(dados.lower())

# 4. Contagem com Spark
negativas_rdd = sc.parallelize(palavras_negativas)

def contar_ocorrencias(termo):
    return (termo, dados.count(termo))

result = negativas_rdd.map(contar_ocorrencias).filter(lambda x: x[1] > 0).collect()

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import random

def marrom_pastel_color_func(word, font_size, position, orientation, random_state=None, **kwargs):

    pastel_browns = [
        (210, 180, 140),
        (222, 184, 135),
        (244, 164, 96),
        (205, 133, 63),
        (193, 154, 107),
        (222, 210, 170),
        (230, 216, 181),
        ]
    r, g, b = random.choice(pastel_browns)
    return f"rgb({r}, {g}, {b})"

frequencies = dict(result)

wordcloud = WordCloud(width=800, height=400, background_color='#f5f0e6').generate_from_frequencies(frequencies)

plt.figure(figsize=(15, 7))
plt.imshow(wordcloud.recolor(color_func=marrom_pastel_color_func), interpolation='bilinear')
plt.axis("off")
plt.title("Nuvem Análise de Sentimentos", color='#5C3A21', fontsize=15)
plt.show()


In [ ]:
# Mostrar a tabela dos termos negativos encontrados
df_result = pd.DataFrame(result, columns=["termo", "frequencia"])
print(df_result.sort_values(by="frequencia", ascending=False))
